# FLIPKART WEBSCRAPINGDATA ANALYSIS....

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Selenium
options = Options()
# options.add_argument("--headless")  # Uncomment to run in headless mode
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def scrape_product_details(product_url):
    driver.get(product_url)
    
    # Extract product details
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Modify these selectors based on the product page structure
    name = soup.find('span', class_='VU-ZEz').text.strip() if soup.find('span', class_='VU-ZEz') else 'N/A'
    price = soup.find('div', class_="Nx9bqj CxhGGd yKS4la").text.strip() if soup.find('div', class_="Nx9bqj CxhGGd yKS4la") else 'N/A'
    discount = soup.find('div', class_="yRaY8j A6+E6v yKS4la").text.strip() if soup.find('div', class_="yRaY8j A6+E6v yKS4la") else 'N/A'
    review = soup.find('span', class_='Wphh3N').text.strip() if soup.find('span', class_='Wphh3N') else 'N/A'
    warranty = soup.find('div', class_='zIL+eP').text.strip() if soup.find('div', class_='zIL+eP') else 'N/A'
   
    return {
        'Product Name': name,
        'Price': price,
        'Discount': discount,
        'Review': review,
        'Warranty': warranty
    }

def scrape_books(url):
    book_data = []
    page_count = 0  # Start page counter at 0
    
    while url and page_count < 30:  # Scrape up to 30 pages
        driver.get(url)
        
        # Scroll down to load more books
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait a bit for content to load
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        books = soup.find_all('div', class_='tUxRFH')

        for book in books:
            try:
                # Get product link
                product_link = book.find('a', class_='CGtC98')['href']
                full_product_url = 'https://www.flipkart.com' + product_link

                # Scrape product details
                product_details = scrape_product_details(full_product_url)

                # Print details
                print(product_details)
                book_data.append(product_details)

            except Exception as e:
                print(f"Error processing book: {e}")

        page_count += 1  # Increment page counter
        
        # Find the next page URL
        next_page = soup.find('a', class_='_9QVEpD')
        if next_page and 'href' in next_page.attrs:
            url = 'https://www.flipkart.com' + next_page['href']
        else:
            url = None  # End loop if there's no next page

    # Convert list to DataFrame
    tv = pd.DataFrame(book_data)
    return tv

# Start scraping from the first page
tv = scrape_books('https://www.flipkart.com/search?q=tv&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off')
driver.quit()  # Close the browser when done

# Save the data to a CSV file
tv.to_csv('scraped_data.csv', index=False)


{'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'}
{'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'}
{'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,94

In [296]:
tv

,Product_Name,Price,Discount,Review,Warranty
0,XElectron 60 cm (24 inch) HD Ready LED TV (24...,"₹5,990","₹13,999",777 Ratings & 155 Reviews,1 Year warranty from the date of purchase. For...
1,Reliance 81 cm (32 inch) HD Ready LED Smart An...,"₹10,745","₹21,490",50 Ratings & 1 Reviews,1 Year Warranty will be provided by company fr...
2,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...,"₹12,490","₹18,900","1,47,714 Ratings & 10,942 Reviews",1 Year Comprehensive Warranty on Product and 1...
3,Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...,"₹12,999","₹24,999","1,68,634 Ratings & 13,109 Reviews",1 Year Warranty on Product and 1 Year Addition...
4,Reliance 81.01 cm (32 inch) HD Ready LED Smart...,"₹10,990","₹21,490",392 Ratings & 36 Reviews,1 Year Comprehensive On Site Warranty from the...
...,...,...,...,...,...
715,realme TechLife CineSonic Q 140 cm (55 inch) Q...,"₹29,999","₹66,999","1,895 Ratings & 205 Reviews",1 Year WarrantyKnow More
716,realme TechLife CineSonic Q 109 cm (43 inch) Q...,"₹24,999","₹45,999","1,895 Ratings & 205 Reviews",1 Year WarrantyKnow More
717,LG UR7500 108 cm (43 inch) Ultra HD (4K) LED S...,"₹29,990","₹49,990","33,759 Ratings & 3,219 Reviews",1 Year Standard Warranty on the Product from t...
718,Infinix 80 cm (32 inch) HD Ready LED Smart Goo...,"₹9,999","₹16,999",558 Ratings & 69 Reviews,1 Year Warranty on ProductKnow More


In [297]:
tv=pd.read_csv('scraped_data.csv')
tv

,Product Name,Price,Discount,Review,Warranty
0,XElectron 60 cm (24 inch) HD Ready LED TV (24...,"₹5,990","₹13,999",777 Ratings & 155 Reviews,1 Year warranty from the date of purchase. For...
1,Reliance 81 cm (32 inch) HD Ready LED Smart An...,"₹10,745","₹21,490",50 Ratings & 1 Reviews,1 Year Warranty will be provided by company fr...
2,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...,"₹12,490","₹18,900","1,47,714 Ratings & 10,942 Reviews",1 Year Comprehensive Warranty on Product and 1...
3,Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...,"₹12,999","₹24,999","1,68,634 Ratings & 13,109 Reviews",1 Year Warranty on Product and 1 Year Addition...
4,Reliance 81.01 cm (32 inch) HD Ready LED Smart...,"₹10,990","₹21,490",392 Ratings & 36 Reviews,1 Year Comprehensive On Site Warranty from the...
...,...,...,...,...,...
715,realme TechLife CineSonic Q 140 cm (55 inch) Q...,"₹29,999","₹66,999","1,895 Ratings & 205 Reviews",1 Year WarrantyKnow More
716,realme TechLife CineSonic Q 109 cm (43 inch) Q...,"₹24,999","₹45,999","1,895 Ratings & 205 Reviews",1 Year WarrantyKnow More
717,LG UR7500 108 cm (43 inch) Ultra HD (4K) LED S...,"₹29,990","₹49,990","33,759 Ratings & 3,219 Reviews",1 Year Standard Warranty on the Product from t...
718,Infinix 80 cm (32 inch) HD Ready LED Smart Goo...,"₹9,999","₹16,999",558 Ratings & 69 Reviews,1 Year Warranty on ProductKnow More


In [298]:
tv.shape

(720, 5)

In [300]:
tv.isnull().sum()

Product Name     0
Price            1
Discount         1
Review          30
Warranty        15
dtype: int64

In [ ]:
## DATA CLEANING

In [328]:
import re
import pandas as pd

# Sample data
data = [
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': 'N/A', 'Discount': 'N/A', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '(530)', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Coocaa 80 cm (32 inch) HD Ready LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(32S3U Plus)', 'Price': '₹8,999', 'Discount': '₹19,990', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Mi by Xiaomi A series 80 cm (32 inch) HD Ready LED Smart Google TV with HD |Dolby Audio | DTS:HD | Vivid Picture Engine', 'Price': '₹11,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 138 cm (55 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA55DUE76AKLXL)', 'Price': '₹42,990', 'Discount': '₹66,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Thomson Phoenix 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV Dolby Vision & Atmos\xa0\xa0(Q43H1110)', 'Price': '₹21,999', 'Discount': '₹31,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 138 cm (55 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA55DUE76AKLXL)', 'Price': '₹42,990', 'Discount': '₹66,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '(530)', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Thomson Phoenix 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV Dolby Vision & Atmos\xa0\xa0(Q43H1110)', 'Price': '₹21,999', 'Discount': '₹31,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 138 cm (55 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA55DUE76AKLXL)', 'Price': '₹42,990', 'Discount': '₹66,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Thomson Phoenix 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV Dolby Vision & Atmos\xa0\xa0(Q43H1110)', 'Price': '₹21,999', 'Discount': '₹31,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 138 cm (55 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA55DUE76AKLXL)', 'Price': '₹42,990', 'Discount': '₹66,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Thomson Phoenix 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV Dolby Vision & Atmos\xa0\xa0(Q43H1110)', 'Price': '₹21,999', 'Discount': '₹31,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'XElectron 60 cm (24 inch) HD Ready LED TV\xa0\xa0(24STV)', 'Price': '₹5,990', 'Discount': '₹13,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(32MP3972)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '50 Ratings\xa0&\xa01 Reviews', 'Warranty': '1 Year Warranty will be provided by company from date of purchase'},
    {'Product Name': 'SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tizen TV with Bezel-Free Design | 300+ Free Channels | PurColor | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA32T4380AKXXL)', 'Price': '₹12,490', 'Discount': '₹18,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional on PanelKnow More'},
    {'Product Name': 'Mi by Xiaomi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with 200+ Free Channels |Dolby Audio | DTS:HD|DTS Virtual: X| Vivid Picture Engine| Dual Band Wi-Fi| Bluetooth 5.0', 'Price': '₹12,999', 'Discount': '₹24,999', 'Review': '1,68,634 Ratings\xa0&\xa013,109 Reviews', 'Warranty': '1 Year Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'Reliance 81.01 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP3972)', 'Price': '₹10,990', 'Discount': '₹21,490', 'Review': '392 Ratings\xa0&\xa036 Reviews', 'Warranty': '1 Year Comprehensive On Site Warranty from the Date of Purchase'},
    {'Product Name': 'XElectron 108 cm (43 inch) Ultra HD (4K) LED Smart Android TV\xa0\xa0(43XETV)', 'Price': '₹19,999', 'Discount': '₹34,999', 'Review': '777 Ratings\xa0&\xa0155 Reviews', 'Warranty': '1 Year warranty from the date of purchase. For more help & support call us at: 8527312304 or mail us at: customercare@xelectron.com'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 108 cm (43 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA43DUE76AKLXL)', 'Price': '₹29,490', 'Discount': '₹46,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'SAMSUNG 108 cm (43 inch) Full HD LED Smart Tizen TV with 300+ Free Channels | HDR | PurColor | Dolby Digital Plus | Hyper Real Picture Engine | Triple Protection | SmartThings App Support | TV Key | Connect Share (HDD) | ConnectShare (USB 2.0)\xa0\xa0(UA43T5450AKXXL)', 'Price': '₹23,990', 'Discount': '₹37,900', 'Review': '1,47,714 Ratings\xa0&\xa010,942 Reviews', 'Warranty': '1 Year Comprehensive Warranty on Product and 1 Year Additional Warranty on PanelKnow More'},
    {'Product Name': 'TCL L4B 79.97 cm (32 inch) HD Ready LED Smart Android TV 2024 Edition with Metallic Bezel Less and Chromecast built-in\xa0\xa0(32L4B)', 'Price': '₹10,490', 'Discount': '₹20,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'iFFALCON by TCL U64 139 cm (55 inch) Ultra HD (4K) LED Smart Google TV with 24W Dolby Audio, HDR10\xa0\xa0(iFF55U64)', 'Price': '₹26,999', 'Discount': '₹73,990', 'Review': '67,052 Ratings\xa0&\xa07,515 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Reliance 108 cm (43 inch) QLED Ultra HD (4K) Smart WebOS TV Fremless Design and Magic Voice Remote.\xa0\xa0(WebOS TV RGT43WB3972UHD)', 'Price': '₹21,990', 'Discount': '₹51,990', 'Review': '(609)', 'Warranty': '1 Year Warranty On Product'},
    {'Product Name': 'STARSHINE 108 cm (43 inch) Full HD LED Smart Google TV with Unlimited OTT Apps, Voice Search, Voice Command Remote, Google Assistant Easy Setup, Netflix, Amazon Prime, Web Browser, AppleTv, Hotstar, YouTube, Mobile Mirroring\xa0\xa0(108 cm (43") Smart GOOGLE TV with 3 Years Warranty and Google Assistant)', 'Price': '₹21,899', 'Discount': '₹31,999', 'Review': 'N/A', 'Warranty': '3 Years Domestic Warranty from the Date of Purchase from Starshine'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition\xa0\xa0(32Y1S /32Y1A)', 'Price': '₹8,499', 'Discount': '₹16,999', 'Review': '48,479 Ratings\xa0&\xa05,109 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'HUIDI 80 cm (32 Inch) HD Ready LED Smart Android Based TV 2024 Edition with With Game Mode\xa0\xa0(HD4FS PRO)', 'Price': '₹7,499', 'Discount': '₹18,999', 'Review': '1,776 Ratings\xa0&\xa0153 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG 32LMBPTC 80 cm (32 inch) HD Ready LED Smart WebOS TV with Quad Core Processor, Active HDR, 60 Hz Refresh Rate, DTS Virtual:X, Dolby Audio\xa0\xa0(32LM563BPTC)', 'Price': '₹13,490', 'Discount': '₹24,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL S5500 79.97 cm (32 inch) Full HD LED Smart Google TV 2024 Edition with 1.5 GB RAM + 16 GB ROM\xa0\xa0(32S5500)', 'Price': '₹11,990', 'Discount': '₹23,990', 'Review': '22,685 Ratings\xa0&\xa01,898 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Reliance 81 cm (32 inch) HD Ready LED Smart Android TV with Android 12, 24 W Dolby Audio, Frameless Design\xa0\xa0(RE32MP39)', 'Price': '₹10,745', 'Discount': '₹21,490', 'Review': '3 Ratings\xa0&\xa00 Reviews', 'Warranty': '1 Year Comprehensive Warranty'},
    {'Product Name': 'STARSHINE 127 cm (50 inch) Ultra HD (4K) LED Smart WebOS TV\xa0\xa0(ATPL-5800)', 'Price': '₹28,793', 'Discount': '₹42,999', 'Review': 'N/A', 'Warranty': 'NA'},
    {'Product Name': 'MOTOROLA EnvisionX 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision & Dolby Atmos\xa0\xa0(43UHDGQMWSTQ)', 'Price': '₹21,699', 'Discount': '₹51,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'LG 32LQBPTA 80 cm (32 inch) HD Ready LED Smart WebOS TV with Alpha5 Gen5 AI Processor, ThinQ AI, AI Sound, AI Acoustic Tuning, 60 Hz Refresh Rate, HDR10, Game Optimizer, Magic Remote Control Ready?\xa0\xa0(32LQ643BPTA)', 'Price': '₹14,990', 'Discount': '₹19,990', 'Review': '21,185 Ratings\xa0&\xa01,405 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'TCL C69B 139 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Atmos 35W, ONKYO 2.1ch with Subwoofer\xa0\xa0(55C69B)', 'Price': '₹36,990', 'Discount': '₹1,20,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'KODAK Special Edition 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Design\xa0\xa0(32SE5001BL)', 'Price': '₹8,499', 'Discount': '₹14,999', 'Review': '5,745 Ratings\xa0&\xa0579 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) HD Ready LED Smart Linux TV 2024 Edition with YouTube, Prime Video and Eyecare Mode\xa0\xa0(D32H1COC)', 'Price': '₹9,999', 'Discount': '₹17,990', 'Review': '2,189 Ratings\xa0&\xa0211 Reviews', 'Warranty': '1 year Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'REDMI 108 cm (43 inch) Ultra HD (4K) LED Smart FireTv OS 7 TV 2024 Edition with 12000+ Apps| Alexa Voice Control| AirPlay 2| HDR 10 with HLG| Vivid Picture Engine| 24W with Dolby Audio| DTS: HD| DTS Virtual: X\xa0\xa0(L43MA-FVIN)', 'Price': '₹21,999', 'Discount': '₹42,999', 'Review': '8,945 Ratings\xa0&\xa0553 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'MOTOROLA 80 cm (32 inch) HD Ready LED Smart Google TV\xa0\xa0(32HDGDMBSXP)', 'Price': '₹11,999', 'Discount': '₹20,000', 'Review': '7,891 Ratings\xa0&\xa0771 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Foxsky 80 cm (32 inch) HD Ready LED Smart Android TV\xa0\xa0(32 FSELS PRO)', 'Price': '₹7,699', 'Discount': '₹22,499', 'Review': '3,717 Ratings\xa0&\xa0720 Reviews', 'Warranty': '1 Year Standard Manufacturer Warranty from SkyWallKnow More'},
    {'Product Name': 'Coocaa 108 cm (43 inch) Full HD LED Smart Coolita TV 2024 Edition with Dolby Audio and Eye Care Technology\xa0\xa0(43C3U Plus)', 'Price': '₹14,999', 'Discount': '₹29,999', 'Review': '22,568 Ratings\xa0&\xa02,672 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(55UHDGQMBSGQ)', 'Price': '₹33,999', 'Discount': '₹1,19,980', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 165 cm (65 inch) QLED Ultra HD (4K) Smart Google TV with Dolby Vision and Dolby Atmos\xa0\xa0(65UHDGQMBSGQ)', 'Price': '₹43,999', 'Discount': '₹1,40,580', 'Review': '9,905 Ratings\xa0&\xa0987 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 139 cm (55 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(55UR7500PSC)', 'Price': '₹42,990', 'Discount': '₹71,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'SAMSUNG New D Series Brighter Crystal 4K Vision Pro (2024 Edition) 138 cm (55 inch) Ultra HD (4K) LED Smart Tizen TV with 4K Upscaling | 300+ Free Channels | Multiple Voice Assistance Remote | Purcolor | HDR 10+ | Auto Game Mode | Q-Symphony | Knox Security\xa0\xa0(UA55DUE76AKLXL)', 'Price': '₹42,990', 'Discount': '₹66,900', 'Review': '36,365 Ratings\xa0&\xa02,899 Reviews', 'Warranty': '2 Year Warranty (1 Year Standard Warranty + 1 Year additional warranty on Panel)Know More'},
    {'Product Name': 'TCL C69B 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV with 30W Dolby Atmos\xa0\xa0(43C69B)', 'Price': '₹26,990', 'Discount': '₹60,990', 'Review': '5,013 Ratings\xa0&\xa0554 Reviews', 'Warranty': '2 Years Comprehensive Warranty on ProductKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 80 cm (32 inch) QLED HD Ready Smart Google TV with 300 nits Brightness & 20w Sound\xa0\xa0(32HDGQMWSTQ)', 'Price': '₹13,799', 'Discount': '₹20,999', 'Review': '3,583 Ratings\xa0&\xa0309 Reviews', 'Warranty': '1 Year Comprehensive WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV 2024 Edition\xa0\xa0(43UHDGDRVS3E)', 'Price': '₹22,999', 'Discount': '₹41,999', 'Review': '1,208 Ratings\xa0&\xa0104 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Acer V Series 80 cm (32 inch) QLED HD Ready Smart Google TV 30W Dolby Audio, 1.5GB RAM, 16GB Storage\xa0\xa0(AR32GR2841VQD)', 'Price': '₹12,999', 'Discount': '₹23,999', 'Review': '4,027 Ratings\xa0&\xa0397 Reviews', 'Warranty': '2 Years Comprehensive Warranty from the Date of PurchaseKnow More'},
    {'Product Name': 'HUIDI 60 cm (24 inch) HD Ready LED TV 2024 Edition\xa0\xa0(HD24D1M19)', 'Price': '₹4,999', 'Discount': '₹10,999', 'Review': '7,677 Ratings\xa0&\xa0818 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'MOTOROLA EnvisionX 109 cm (43 inch) Ultra HD (4K) LED Smart Google TV with Inbuilt Box Speakers\xa0\xa0(43UHDGDMBSXP)', 'Price': '₹25,299', 'Discount': '₹49,990', 'Review': '5,703 Ratings\xa0&\xa0537 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Daiwa 80 cm (32 inch) QLED HD Ready Smart Google TV Chromecast, Dolby Audio,Kids and Family feature\xa0\xa0(32G1Q)', 'Price': '₹12,499', 'Discount': '₹25,990', 'Review': '696 Ratings\xa0&\xa0150 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Mi by Xiaomi X Series 125 cm (50 inch) Ultra HD (4K) LED Smart Google TV with 4K Dolby Vision | HDR 10 | Dolby Audio |,DTS X | DTS Virtual: X | Vivid Picture Engine', 'Price': '₹29,999', 'Discount': '₹44,999', 'Review': '1,22,892 Ratings\xa0&\xa011,258 Reviews', 'Warranty': '1 Year Warranty on Product and 2 Years Warranty on PanelKnow More'},
    {'Product Name': 'Acer I PRO Series 80.01 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition with Android 14 (Black) | 16GB Storage | 30W Dolby Audio\xa0\xa0(AR32HDIGU2841AT)', 'Price': '₹12,599', 'Discount': '₹22,999', 'Review': '26,058 Ratings\xa0&\xa02,825 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'Thomson Phoenix 80 cm (32 inch) QLED HD Ready Smart Android TV 48W Sound Output\xa0\xa0(Q32H1111)', 'Price': '₹11,999', 'Discount': '₹19,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product & 6 Months Warranty on AccessoriesKnow More'},
    {'Product Name': 'InnoQ Spectra 80 cm (32 inch) HD Ready LED Smart Android TV with 30W Boom Speakers| 1000+ Smart Apps - Games, Mobile Screen Connect, Pixel Colour Enhancer\xa0\xa0(32S-SPECTRA/1)', 'Price': '₹7,990', 'Discount': '₹29,990', 'Review': '3,491 Ratings\xa0&\xa0395 Reviews', 'Warranty': '1 Year on TV | Free Extended Warranty'},
    {'Product Name': 'Foxsky 80 cm (32 inch) QLED HD Ready Smart Google TV Dolby Audio ( 2024)\xa0\xa0(32FS-GOOGLE LED TV)', 'Price': '₹8,999', 'Discount': '₹26,499', 'Review': '605 Ratings\xa0&\xa093 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 140 cm (55 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(55UHDGQRWSGQ)', 'Price': '₹29,999', 'Discount': '₹66,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'realme TechLife CineSonic Q 109 cm (43 inch) QLED Ultra HD (4K) Smart Google TV\xa0\xa0(43UHDGQRVS3Q)', 'Price': '₹24,999', 'Discount': '₹45,999', 'Review': '1,895 Ratings\xa0&\xa0205 Reviews', 'Warranty': '1 Year WarrantyKnow More'},
    {'Product Name': 'LG UR7500 108 cm (43 inch) Ultra HD (4K) LED Smart WebOS TV with Alpha5 AI Processor 4K Gen6, AI Picture Pro, 4K Upscaling, 60 Hz Refresh Rate, Game Optimizer, Magic Remote Capability, 5-Year Guaranteed with New OS Experience\xa0\xa0(43UR7500PSC)', 'Price': '₹29,990', 'Discount': '₹49,990', 'Review': '33,759 Ratings\xa0&\xa03,219 Reviews', 'Warranty': '1 Year Standard Warranty on the Product from the Date of Purchase. (T&C Apply)Know More'},
    {'Product Name': 'Infinix 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition\xa0\xa0(32GH1)', 'Price': '₹9,999', 'Discount': '₹16,999', 'Review': '558 Ratings\xa0&\xa069 Reviews', 'Warranty': '1 Year Warranty on ProductKnow More'},
    {'Product Name': 'Thomson Phoenix 108 cm (43 inch) QLED Ultra HD (4K) Smart Google TV Dolby Vision & Atmos\xa0\xa0(Q43H1110)', 'Price': '₹21,999', 'Discount': '₹31,999', 'Review': '22,681 Ratings\xa0&\xa04,063 Reviews', 'Warranty': '1 Year Warranty on Product and 6 Months Warranty on AccessoriesKnow More'},
]   
# Function to extract brand and TV inches
def extract_brand_and_inches(product_name):
    # Extract brand (first word)
    brand = product_name.split()[0]
    
    # Extract TV inches (first number in inches)
    match = re.search(r'(\d+)\s*inch', product_name, re.IGNORECASE)
    inches = match.group(1) if match else None
    
    return brand, inches

# Apply extraction function to data
for item in data:
    brand, inches = extract_brand_and_inches(item['Product Name'])
    item['Brand'] = brand
    item['TV Inches'] = inches

# Convert to DataFrame to view results
tv = pd.DataFrame(data)
print(tv[['Product Name', 'Brand', 'TV Inches']])

                                          Product Name      Brand TV Inches
0    XElectron 60 cm (24 inch) HD Ready LED TV  (24...  XElectron        24
1    Reliance 81 cm (32 inch) HD Ready LED Smart An...   Reliance        32
2    SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...    SAMSUNG        32
3    Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...         Mi        32
4    Reliance 81.01 cm (32 inch) HD Ready LED Smart...   Reliance        32
..                                                 ...        ...       ...
715  realme TechLife CineSonic Q 140 cm (55 inch) Q...     realme        55
716  realme TechLife CineSonic Q 109 cm (43 inch) Q...     realme        43
717  LG UR7500 108 cm (43 inch) Ultra HD (4K) LED S...         LG        43
718  Infinix 80 cm (32 inch) HD Ready LED Smart Goo...    Infinix        32
719  Thomson Phoenix 108 cm (43 inch) QLED Ultra HD...    Thomson        43

[720 rows x 3 columns]


## CREATE NEW COLUMNS ( BRAND , TV INCHES ) ^

In [329]:
tv.shape

(720, 7)

In [ ]:
## SPLIT THE REVIEW COLUMN ( RATINGS AND REVIEWS)

In [330]:
# Use str.extract with a regular expression to separate ratings and reviews
tv[['Ratings', 'Reviews']] = tv['Review'].str.extract(r'([\d,]+)\s+Ratings\s*&\s*([\d,]+)\s+Reviews')

# Remove commas from the numbers and convert them to integers
tv['Ratings'] = tv['Ratings'].str.replace(',', '').astype(float)
tv['Reviews'] = tv['Reviews'].str.replace(',', '').astype(float)

# Check the result
print(tv[['Ratings', 'Reviews']])

      Ratings  Reviews
0       777.0    155.0
1        50.0      1.0
2    147714.0  10942.0
3    168634.0  13109.0
4       392.0     36.0
..        ...      ...
715    1895.0    205.0
716    1895.0    205.0
717   33759.0   3219.0
718     558.0     69.0
719   22681.0   4063.0

[720 rows x 2 columns]


## Extract the first warranty year mentioned in each string

In [331]:
# Extract the first warranty year mentioned in each string
tv['Warranty Years'] = tv['Warranty'].str.extract(r'(\d+)\s*Year')

# Convert to integer, replacing NaN with 0 (if no warranty info is found)
tv['Warranty Years'] = tv['Warranty Years'].fillna(0).astype(int)

# Display the result
print(tv[['Warranty', 'Warranty Years']])

                                              Warranty  Warranty Years
0    1 Year warranty from the date of purchase. For...               1
1    1 Year Warranty will be provided by company fr...               1
2    1 Year Comprehensive Warranty on Product and 1...               1
3    1 Year Warranty on Product and 1 Year Addition...               1
4    1 Year Comprehensive On Site Warranty from the...               1
..                                                 ...             ...
715                           1 Year WarrantyKnow More               1
716                           1 Year WarrantyKnow More               1
717  1 Year Standard Warranty on the Product from t...               1
718                1 Year Warranty on ProductKnow More               1
719  1 Year Warranty on Product and 6 Months Warran...               1

[720 rows x 2 columns]


## RENAME THE COLUMNS

In [332]:
# Renaming columns
tv.rename(columns={'Price': 'Discount Price', 'Discount': 'Actual Price'}, inplace=True)

In [333]:
tv.sample(3)

,Product Name,Discount Price,Actual Price,Review,Warranty,Brand,TV Inches,Ratings,Reviews,Warranty Years
181,Daiwa 80 cm (32 inch) QLED HD Ready Smart Goog...,"₹12,499","₹25,990",696 Ratings & 150 Reviews,1 Year WarrantyKnow More,Daiwa,32,696.0,150.0,1
107,STARSHINE 108 cm (43 inch) Full HD LED Smart G...,"₹21,899","₹31,999",N/A,3 Years Domestic Warranty from the Date of Pur...,STARSHINE,43,NaN,NaN,3
244,Reliance 81.01 cm (32 inch) HD Ready LED Smart...,"₹10,990","₹21,490",392 Ratings & 36 Reviews,1 Year Comprehensive On Site Warranty from the...,Reliance,32,392.0,36.0,1


## DROP THE OLD COLUMNS

In [334]:
# Drop the old 'Review' and 'Warranty' columns
tv.drop(columns=['Review', 'Warranty'], inplace=True)

# Display the updated DataFrame to confirm
print(tv.head())

                                        Product Name Discount Price  \
0  XElectron 60 cm (24 inch) HD Ready LED TV  (24...         ₹5,990   
1  Reliance 81 cm (32 inch) HD Ready LED Smart An...        ₹10,745   
2  SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...        ₹12,490   
3  Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...        ₹12,999   
4  Reliance 81.01 cm (32 inch) HD Ready LED Smart...        ₹10,990   

  Actual Price      Brand TV Inches   Ratings  Reviews  Warranty Years  
0      ₹13,999  XElectron        24     777.0    155.0               1  
1      ₹21,490   Reliance        32      50.0      1.0               1  
2      ₹18,900    SAMSUNG        32  147714.0  10942.0               1  
3      ₹24,999         Mi        32  168634.0  13109.0               1  
4      ₹21,490   Reliance        32     392.0     36.0               1  


## CREATE THE COLUMN ( SCREEN TYPE )

In [335]:
import pandas as pd

# Define a function to extract screen type
def extract_screen_type(product_name):
    # Assuming screen type appears as "HD Ready," "Full HD," etc., in Product Name
    types = ["HD Ready", "Full HD", "Ultra HD", "4K" , "QLED"]
    for t in types:
        if t in product_name:
            return t
    return None  # Return None if no screen type found

# Apply the function to create the new column
tv['Screen Type'] = tv['Product Name'].apply(extract_screen_type)

# Display the updated DataFrame with new Screen Type column
print(df.head())


                                        Product_Name  Discount_Price  \
0  XElectron 60 cm (24 inch) HD Ready LED TV  (24...          5990.0   
1  Reliance 81 cm (32 inch) HD Ready LED Smart An...         10745.0   
2  SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...         12490.0   
3  Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...         12999.0   
4  Reliance 81.01 cm (32 inch) HD Ready LED Smart...         10990.0   

   Actual_Price                             Review  \
0       13999.0          777 Ratings & 155 Reviews   
1       21490.0             50 Ratings & 1 Reviews   
2       18900.0  1,47,714 Ratings & 10,942 Reviews   
3       24999.0  1,68,634 Ratings & 13,109 Reviews   
4       21490.0           392 Ratings & 36 Reviews   

                                            Warranty                  Brand  \
0  1 Year warranty from the date of purchase. For...              XElectron   
1  1 Year Warranty will be provided by company fr...               Reliance   
2  1 

In [336]:
# Drop rows where 'Ratings' or 'Reviews' have null values
tv.dropna(subset=['Ratings', 'Reviews'], inplace=True)

# Display the updated DataFrame to confirm
print(tv.isnull().sum())  # This will show you the count of nulls in each column to confirm removal


Product Name      0
Discount Price    0
Actual Price      0
Brand             0
TV Inches         0
Ratings           0
Reviews           0
Warranty Years    0
Screen Type       0
dtype: int64


## REMOVE SYMBOLS

In [337]:
# Remove '₹' symbol and commas from 'Discount Price' and 'Actual Price'
tv['Discount Price'] = tv['Discount Price'].replace({'₹': '', ',': ''}, regex=True)
tv['Actual Price'] = tv['Actual Price'].replace({'₹': '', ',': ''}, regex=True)

# Convert 'Discount Price' and 'Actual Price' to float, handle 'N/A' and non-numeric values
tv['Discount Price'] = pd.to_numeric(tv['Discount Price'], errors='coerce')
tv['Actual Price'] = pd.to_numeric(tv['Actual Price'], errors='coerce')

# Convert 'TV Inches' to numeric, in case it has non-numeric values like 'N/A'
# tv['TV_Inches'] = pd.to_numeric(tv['TV_Inches'], errors='coerce')

# Check the data types again to confirm the conversion
print(tv.dtypes)

# Check for any NaN values to confirm they were handled correctly
print(tv.isna().sum())


Product Name       object
Discount Price    float64
Actual Price      float64
Brand              object
TV Inches          object
Ratings           float64
Reviews           float64
Warranty Years      int32
Screen Type        object
dtype: object
Product Name      0
Discount Price    1
Actual Price      1
Brand             0
TV Inches         0
Ratings           0
Reviews           0
Warranty Years    0
Screen Type       0
dtype: int64


In [339]:
print(tv.columns)

Index(['Product Name', 'Discount Price', 'Actual Price', 'Brand', 'TV Inches',
       'Ratings', 'Reviews', 'Warranty Years', 'Screen Type'],
      dtype='object')


In [338]:
# Fill missing values in the 'Discount Price' column with the mean value of that columns
tv['Discount Price'] = tv['Discount Price'].fillna(tv['Discount Price'].mean())

# Fill missing values in the 'Actual Price' column with the mean value of that column
tv['Actual Price'] = tv['Actual Price'].fillna(tv['Actual Price'].mean())

# Check for any NaN values to confirm they were handled correctly
print(tv.isna().sum())

Product Name      0
Discount Price    0
Actual Price      0
Brand             0
TV Inches         0
Ratings           0
Reviews           0
Warranty Years    0
Screen Type       0
dtype: int64


## ONE SAMPLE T TEST

In [340]:
import pandas as pd
import scipy.stats as stats

# Sample data (replace with your actual dataset)

# Set the population mean you want to test against
population_mean_discount_price = 10000  # Change this to your desired value

# Convert 'Discount Price' to a numeric type (removing symbols like ₹ and commas if any)
tv['Discount Price'] = tv['Discount Price'].replace({'₹': '', ',': ''}, regex=True).astype(float)

# Perform the one-sample t-test on 'Discount Price'
t_stat_discount, p_value_discount = stats.ttest_1samp(tv['Discount Price'].dropna(), population_mean_discount_price)

# Print the results
print(f"Discount Price - T-statistic: {t_stat_discount}, P-value: {p_value_discount}")


Discount Price - T-statistic: 20.2683156350507, P-value: 1.215474330280995e-71


## Interpretation:
### If the p-value is less than 0.05, it means the mean discount price is significantly different from ₹10,000.
### If the p-value is greater than 0.05, it means there is not enough evidence to reject the null hypothesis, and the mean discount price is not significantly different from ₹10,000.

In [235]:
# Drop the 'Ratings' column from the DataFrame
# tv = tv.drop(columns=['Ratings'])


In [342]:
tv

,Product Name,Discount Price,Actual Price,Brand,TV Inches,Ratings,Reviews,Warranty Years,Screen Type
0,XElectron 60 cm (24 inch) HD Ready LED TV (24...,5990.0,13999.0,XElectron,24,777.0,155.0,1,HD Ready
1,Reliance 81 cm (32 inch) HD Ready LED Smart An...,10745.0,21490.0,Reliance,32,50.0,1.0,1,HD Ready
2,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...,12490.0,18900.0,SAMSUNG,32,147714.0,10942.0,1,HD Ready
3,Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...,12999.0,24999.0,Mi,32,168634.0,13109.0,1,HD Ready
4,Reliance 81.01 cm (32 inch) HD Ready LED Smart...,10990.0,21490.0,Reliance,32,392.0,36.0,1,HD Ready
...,...,...,...,...,...,...,...,...,...
715,realme TechLife CineSonic Q 140 cm (55 inch) Q...,29999.0,66999.0,realme,55,1895.0,205.0,1,Ultra HD
716,realme TechLife CineSonic Q 109 cm (43 inch) Q...,24999.0,45999.0,realme,43,1895.0,205.0,1,Ultra HD
717,LG UR7500 108 cm (43 inch) Ultra HD (4K) LED S...,29990.0,49990.0,LG,43,33759.0,3219.0,1,Ultra HD
718,Infinix 80 cm (32 inch) HD Ready LED Smart Goo...,9999.0,16999.0,Infinix,32,558.0,69.0,1,HD Ready


In [224]:
#pip install mysql-connector-python


In [343]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

# MySQL database connection
config = {
    "user": "root",
    "password": "Admin@123",
    "host": "localhost",
    "database": "flipkart_data_db",
    "raise_on_warnings": True
}

try:
    # Connect to the database
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

    # Insert data into the Products table
    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO flipkart_data (Product_Name, Discount_Price, Actual_Price, Brand, TV_Inches, Reviews, Warranty_Years, Screen_Type)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['Product_Name'],
            row['Discount_Price'],
            row['Actual_Price'],
            row['Brand'],
            row['TV_Inches'],
            row['Reviews'],
            row['Warranty_Years'],
            row['Screen_Type']
        ))

    # Commit the transaction
    connection.commit()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    cursor.close()
    connection.close()


In [344]:
tv.head()

,Product Name,Discount Price,Actual Price,Brand,TV Inches,Ratings,Reviews,Warranty Years,Screen Type
0,XElectron 60 cm (24 inch) HD Ready LED TV (24...,5990.0,13999.0,XElectron,24,777.0,155.0,1,HD Ready
1,Reliance 81 cm (32 inch) HD Ready LED Smart An...,10745.0,21490.0,Reliance,32,50.0,1.0,1,HD Ready
2,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...,12490.0,18900.0,SAMSUNG,32,147714.0,10942.0,1,HD Ready
3,Mi by Xiaomi A Series 80 cm (32 inch) HD Ready...,12999.0,24999.0,Mi,32,168634.0,13109.0,1,HD Ready
4,Reliance 81.01 cm (32 inch) HD Ready LED Smart...,10990.0,21490.0,Reliance,32,392.0,36.0,1,HD Ready


In [345]:
# Save the cleaned dataframe to a new CSV file
tv.to_csv('cleaned_web_data5.csv', index=False)